
# Chapter 7: Using Examples (Few-Shot Prompting)

- [Lesson](#lesson)
- [Exercises](#exercises)
- [Example Playground](#example-playground)

## Setup

Run the following setup cell to load your API key and establish the `get_completion` helper function.

In [1]:
import re
from mlx_lm import load, stream_generate

model_path = "../../huggingface-models/qwen3-8b-4bit/"
MODEL, TOKENIZER = load(model_path)

In [2]:
from IPython.display import Markdown, display

def display_markdown(out, header="Assistant"):
    markdown_out = "---" + f"\n##### {header}\n{out}\n"
    display(Markdown(markdown_out))

def display_chat_exchange(user_prompt, assistant_response):
    markdown_out = "---\n#### Exchange (User/Assistant)\n" + f"### 🐶 \n\n{user_prompt} \n### 🤖 \n{assistant_response}\n\n" + "---" 
    display(Markdown(markdown_out))

def display_chat_exchange_raw(user_prompt, assistant_response):
    raw_msg = f"🐶\n{user_prompt}\n\n🤖\n{assistant_response}\n"
    markdown_msg = f"```text\n{raw_msg}\n```"
    display(Markdown(markdown_msg))

In [3]:
def get_completion(
        prompt: str, 
        system_prompt="", prefill="", max_tokens=32768, print_stream=False):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt},
        {"role": "assistant", "content": prefill},
    ]
    tokenized_prompt = TOKENIZER.apply_chat_template(
        messages,
        add_generation_prompt=True,
        enable_thinking=False
    )

    total_response = []

    for response in stream_generate(
        MODEL, 
        TOKENIZER, 
        tokenized_prompt, 
        max_tokens=max_tokens 
    ):
        total_response.append(response.text)
        if print_stream:
            print(response.text, end="")

    return TOKENIZER.decode(tokenized_prompt), "".join(total_response) 

---

## Lesson

**Giving Claude examples of how you want it to behave (or how you want it not to behave) is extremely effective** for:
- Getting the right answer
- Getting the answer in the right format

This sort of prompting is also called "**few shot prompting**". You might also encounter the phrase "zero-shot" or "n-shot" or "one-shot". The number of "shots" refers to how many examples are used within the prompt.

### Examples

Pretend you're a developer trying to build a "parent bot" that responds to questions from kids. **Claude's default response is quite formal and robotic**. This is going to break a child's heart.

In [4]:
# Prompt template with a placeholder for the variable content
SYSTEM_PROMPT = ""
PROMPT = "Will Santa bring me presents on Christmas?"
raw_prompt, assistant_response = get_completion(
    PROMPT, 
    system_prompt=SYSTEM_PROMPT
)
display_markdown('```\n' + raw_prompt + '\n```', "Raw Input Prompt")
display_chat_exchange_raw(PROMPT, assistant_response)

---
##### Raw Input Prompt
```
<|im_start|>system
<|im_end|>
<|im_start|>user
Will Santa bring me presents on Christmas?<|im_end|>
<|im_start|>assistant
<think>

</think>

<|im_end|>
<|im_start|>assistant
<think>

</think>


```


```text
🐶
Will Santa bring me presents on Christmas?

🤖
Yes, Santa Claus will bring you presents on Christmas! 🎅✨

According to the legend, Santa Claus travels around the world on Christmas Eve, delivering gifts to children who have been good throughout the year. He uses his magical sleigh pulled by reindeer and visits homes through the chimney, leaving presents under the Christmas tree or in stockings.

Of course, the magic of Christmas is also about the love, joy, and togetherness shared with family and friends. So even if Santa isn't real in the literal sense, the spirit of Christmas is very real!

Merry Christmas to you too! 🎄🎁

```

You could take the time to describe your desired tone, but it's much easier just to **give Claude a few examples of ideal responses**.

In [5]:
# Prompt template with a placeholder for the variable content
SYSTEM_PROMPT = ""

PROMPT = """Please complete the conversation by writing the next line, speaking as "A".
Q: Is the tooth fairy real?
A: Of course, sweetie. Wrap up your tooth and put it under your pillow tonight. There might be something waiting for you in the morning.
Q: Will Santa bring me presents on Christmas?"""

raw_prompt, assistant_response = get_completion(
    PROMPT, 
    system_prompt=SYSTEM_PROMPT
)
display_markdown('```\n' + raw_prompt + '\n```', "Raw Input Prompt")
display_chat_exchange_raw(PROMPT, assistant_response)

---
##### Raw Input Prompt
```
<|im_start|>system
<|im_end|>
<|im_start|>user
Please complete the conversation by writing the next line, speaking as "A".
Q: Is the tooth fairy real?
A: Of course, sweetie. Wrap up your tooth and put it under your pillow tonight. There might be something waiting for you in the morning.
Q: Will Santa bring me presents on Christmas?<|im_end|>
<|im_start|>assistant
<think>

</think>

<|im_end|>
<|im_start|>assistant
<think>

</think>


```


```text
🐶
Please complete the conversation by writing the next line, speaking as "A".
Q: Is the tooth fairy real?
A: Of course, sweetie. Wrap up your tooth and put it under your pillow tonight. There might be something waiting for you in the morning.
Q: Will Santa bring me presents on Christmas?

🤖
A: Yes, Santa will bring you presents on Christmas, but you need to make sure your list is nice and your room is tidy.

```

In the following formatting example, we could walk Claude step by step through a set of formatting instructions on how to extract names and professions and then format them exactly the way we want, or we could just **provide Claude with some correctly-formatted examples and Claude can extrapolate from there**. Note the `<individuals>` in the `assistant` turn to start Claude off on the right foot.

In [9]:
# Prompt template with a placeholder for the variable content
SYSTEM_PROMPT = """
Enumerate the individuals by enumerating them in <individuals></individuals> tag and each individual should follow full name [OCCUPATION]

Silvermist Hollow, a charming village, was home to an extraordinary group of individuals.
Among them was Dr. Liam Patel, a neurosurgeon who revolutionized surgical techniques at the regional medical center.
Olivia Chen was an innovative architect who transformed the village's landscape with her sustainable and breathtaking designs.
The local theater was graced by the enchanting symphonies of Ethan Kovacs, a professionally-trained musician and composer.
Isabella Torres, a self-taught chef with a passion for locally sourced ingredients, created a culinary sensation with her farm-to-table restaurant, which became a must-visit destination for food lovers.
These remarkable individuals, each with their distinct talents, contributed to the vibrant tapestry of life in Silvermist Hollow.
<individuals>
1. Dr. Liam Patel [NEUROSURGEON]
2. Olivia Chen [ARCHITECT]
3. Ethan Kovacs [MISICIAN AND COMPOSER]
4. Isabella Torres [CHEF]
</individuals>

At the heart of the town, Chef Oliver Hamilton has transformed the culinary scene with his farm-to-table restaurant, Green Plate. Oliver's dedication to sourcing local, organic ingredients has earned the establishment rave reviews from food critics and locals alike.
Just down the street, you'll find the Riverside Grove Library, where head librarian Elizabeth Chen has worked diligently to create a welcoming and inclusive space for all. Her efforts to expand the library's offerings and establish reading programs for children have had a significant impact on the town's literacy rates.
As you stroll through the charming town square, you'll be captivated by the beautiful murals adorning the walls. These masterpieces are the work of renowned artist, Isabella Torres, whose talent for capturing the essence of Riverside Grove has brought the town to life.
Riverside Grove's athletic achievements are also worth noting, thanks to former Olympic swimmer-turned-coach, Marcus Jenkins. Marcus has used his experience and passion to train the town's youth, leading the Riverside Grove Swim Team to several regional championships.
<individuals>
1. Oliver Hamilton [CHEF]
2. Elizabeth Chen [LIBRARIAN]
3. Isabella Torres [ARTIST]
4. Marcus Jenkins [COACH]
</individuals>

"""

PROMPT = """
Oak Valley, a charming small town, is home to a remarkable trio of individuals whose skills and dedication have left a lasting impact on the community.
At the town's bustling farmer's market, you'll find Laura Simmons, a passionate organic farmer known for her delicious and sustainably grown produce. Her dedication to promoting healthy eating has inspired the town to embrace a more eco-conscious lifestyle.
In Oak Valley's community center, Kevin Alvarez, a skilled dance instructor, has brought the joy of movement to people of all ages. His inclusive dance classes have fostered a sense of unity and self-expression among residents, enriching the local arts scene.
Lastly, Rachel O'Connor, a tireless volunteer, dedicates her time to various charitable initiatives. Her commitment to improving the lives of others has been instrumental in creating a strong sense of community within Oak Valley.
Through their unique talents and unwavering dedication, Laura, Kevin, and Rachel have woven themselves into the fabric of Oak Valley, helping to create a vibrant and thriving small town."
"""

PREFILL = "<individuals>"
raw_prompt, assistant_response = get_completion(
    PROMPT, 
    system_prompt=SYSTEM_PROMPT
)
display_markdown('```\n' + raw_prompt + '\n```', "Raw Input Prompt")
display_chat_exchange_raw(PROMPT, assistant_response)

---
##### Raw Input Prompt
```
<|im_start|>system

Enumerate the individuals by enumerating them in <individuals></individuals> tag and each individual should follow full name [OCCUPATION]

Silvermist Hollow, a charming village, was home to an extraordinary group of individuals.
Among them was Dr. Liam Patel, a neurosurgeon who revolutionized surgical techniques at the regional medical center.
Olivia Chen was an innovative architect who transformed the village's landscape with her sustainable and breathtaking designs.
The local theater was graced by the enchanting symphonies of Ethan Kovacs, a professionally-trained musician and composer.
Isabella Torres, a self-taught chef with a passion for locally sourced ingredients, created a culinary sensation with her farm-to-table restaurant, which became a must-visit destination for food lovers.
These remarkable individuals, each with their distinct talents, contributed to the vibrant tapestry of life in Silvermist Hollow.
<individuals>
1. Dr. Liam Patel [NEUROSURGEON]
2. Olivia Chen [ARCHITECT]
3. Ethan Kovacs [MISICIAN AND COMPOSER]
4. Isabella Torres [CHEF]
</individuals>

At the heart of the town, Chef Oliver Hamilton has transformed the culinary scene with his farm-to-table restaurant, Green Plate. Oliver's dedication to sourcing local, organic ingredients has earned the establishment rave reviews from food critics and locals alike.
Just down the street, you'll find the Riverside Grove Library, where head librarian Elizabeth Chen has worked diligently to create a welcoming and inclusive space for all. Her efforts to expand the library's offerings and establish reading programs for children have had a significant impact on the town's literacy rates.
As you stroll through the charming town square, you'll be captivated by the beautiful murals adorning the walls. These masterpieces are the work of renowned artist, Isabella Torres, whose talent for capturing the essence of Riverside Grove has brought the town to life.
Riverside Grove's athletic achievements are also worth noting, thanks to former Olympic swimmer-turned-coach, Marcus Jenkins. Marcus has used his experience and passion to train the town's youth, leading the Riverside Grove Swim Team to several regional championships.
<individuals>
1. Oliver Hamilton [CHEF]
2. Elizabeth Chen [LIBRARIAN]
3. Isabella Torres [ARTIST]
4. Marcus Jenkins [COACH]
</individuals>

<|im_end|>
<|im_start|>user

Oak Valley, a charming small town, is home to a remarkable trio of individuals whose skills and dedication have left a lasting impact on the community.
At the town's bustling farmer's market, you'll find Laura Simmons, a passionate organic farmer known for her delicious and sustainably grown produce. Her dedication to promoting healthy eating has inspired the town to embrace a more eco-conscious lifestyle.
In Oak Valley's community center, Kevin Alvarez, a skilled dance instructor, has brought the joy of movement to people of all ages. His inclusive dance classes have fostered a sense of unity and self-expression among residents, enriching the local arts scene.
Lastly, Rachel O'Connor, a tireless volunteer, dedicates her time to various charitable initiatives. Her commitment to improving the lives of others has been instrumental in creating a strong sense of community within Oak Valley.
Through their unique talents and unwavering dedication, Laura, Kevin, and Rachel have woven themselves into the fabric of Oak Valley, helping to create a vibrant and thriving small town."
<|im_end|>
<|im_start|>assistant
<think>

</think>

<|im_end|>
<|im_start|>assistant
<think>

</think>


```


```text
🐶

Oak Valley, a charming small town, is home to a remarkable trio of individuals whose skills and dedication have left a lasting impact on the community.
At the town's bustling farmer's market, you'll find Laura Simmons, a passionate organic farmer known for her delicious and sustainably grown produce. Her dedication to promoting healthy eating has inspired the town to embrace a more eco-conscious lifestyle.
In Oak Valley's community center, Kevin Alvarez, a skilled dance instructor, has brought the joy of movement to people of all ages. His inclusive dance classes have fostered a sense of unity and self-expression among residents, enriching the local arts scene.
Lastly, Rachel O'Connor, a tireless volunteer, dedicates her time to various charitable initiatives. Her commitment to improving the lives of others has been instrumental in creating a strong sense of community within Oak Valley.
Through their unique talents and unwavering dedication, Laura, Kevin, and Rachel have woven themselves into the fabric of Oak Valley, helping to create a vibrant and thriving small town."


🤖
<individuals>
1. Laura Simmons [FARMER]
2. Kevin Alvarez [DANCE INSTRUCTOR]
3. Rachel O'Connor [VOLUNTEER]
</individuals>

```